In [125]:
import datetime as datetime
import pandas as pd
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import requests

short_start = "2019-2-01"
med_start = "2019-1-01"
long_start = "2018-01-01"
short_end = "2019-3-01"
med_end = "2019-3-01"
long_end = "2019-3-01"

drop_list = ['XRP' , 'XLM' , 'BCH' , 'LTC' , 'ADA' , 'TRX' , 'EOS' , 'BSV']
long_drop = ['currency' , 'platform' , 'application']

tfs = ['short' , 'med' , 'long']

df_all = dict.fromkeys(tfs)
df_benchmarks = dict.fromkeys(tfs)
returns_all = dict.fromkeys(tfs)
returns_benchmarks = dict.fromkeys(tfs)
results = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
net_return = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
ann_factor = dict.fromkeys(tfs)
total_return = dict.fromkeys(tfs)
percent_sd = dict.fromkeys(tfs)
summary = dict.fromkeys(tfs)
corr = dict.fromkeys(tfs)
betas = dict.fromkeys(tfs)
beta_results = dict.fromkeys(tfs)
total_return_excel = dict.fromkeys(tfs)
sortino_df = dict.fromkeys(tfs)
sortino = dict.fromkeys(tfs)
average_return = dict.fromkeys(tfs)

data_bletchley = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\\bletchley_history.xlsx' , index_col= 0)
data_coins = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bletchley.xlsx' , index_col=0)
data_bitcoin = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bitcoin.xlsx' , index_col=0)
data_bletchley['ERC20'] = data_bletchley['ethereum']
data_bletchley = data_bletchley.drop('ethereum' , axis=1)
data_bletch = data_bletchley

In [127]:
data_finance = data_bletch.fillna(method='ffill')
data_finance = data_finance.dropna()
index_data = data_finance
data_coins = data_coins.fillna(method='ffill')
data_coins = data_coins.dropna()

bitcoin_data = data_bitcoin.fillna(method='ffill')
bitcoin_data = bitcoin_data.dropna()
bitcoin_data.index = (pd.to_datetime(bitcoin_data.index.values,unit='ms'))
bitcoin_data.index = bitcoin_data.index.round('d')

index_data.index = (pd.to_datetime(index_data.index.values,unit='ms'))
index_data.index = index_data.index.round('d')
df = index_data.join(data_coins, how='inner')
df = df.join(bitcoin_data,how='inner')
df

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BTC
2018-11-08,740.953740,637.900186,110.287135,311.303644,40.569745,27.086623,24.329500,73.888661,216.81275,0.53424,0.25491,53.76466,621.38894,0.02373,5.61838,0.07791,6455.12727,6522.27
2018-11-09,721.565444,618.797851,106.479184,303.007814,39.834951,25.967035,23.624929,71.826111,210.95792,0.49365,0.25513,52.15122,573.79060,0.02300,5.49833,0.07607,102.76530,6442.60
2018-11-10,713.346165,609.780474,103.749804,299.376551,39.246942,25.732236,23.221154,70.634206,209.82396,0.50222,0.25729,51.96081,542.83496,0.02314,5.39668,0.07439,70.38530,6386.13
2018-11-11,716.586940,610.434094,104.335381,300.675133,39.365448,25.905354,23.253192,70.736600,212.30790,0.50725,0.26331,52.21095,553.37638,0.02296,5.41311,0.07517,88.41436,6413.63
2018-11-12,719.176084,608.269451,102.901564,301.480361,39.486859,26.000422,22.972163,69.884891,211.27900,0.50607,0.27489,51.05328,527.57848,0.02282,5.45308,0.07639,114.92158,6411.76
2018-11-13,716.724558,611.223194,101.636233,300.550535,39.208126,26.162344,22.709419,68.951585,210.39299,0.51678,0.26643,50.49293,661.71065,0.02231,5.40342,0.07564,725.37542,6373.19
2018-11-14,708.465071,596.955295,98.718815,296.715644,38.893893,25.491212,22.114844,66.856327,203.01291,0.49008,0.24125,47.86484,471.97180,0.02056,5.11448,0.06933,190.26777,6351.24
2018-11-15,639.335345,531.505568,86.586428,267.333560,35.057002,22.916454,19.560126,59.560033,183.47111,0.47568,0.23364,43.58745,415.19188,0.01958,4.70496,0.06549,173.89303,5736.15
2018-11-16,628.338127,520.364846,84.732955,262.641013,34.171254,22.849624,19.211085,58.138167,181.35113,0.47954,0.24061,43.66456,355.11370,0.01895,4.71277,0.06322,89.77542,5645.32
2018-11-17,612.375185,503.651023,82.562568,255.850709,33.395493,22.059975,18.680848,56.552413,175.09354,0.47487,0.23985,42.16066,294.76856,0.01903,4.58613,0.06151,113.02762,5578.58


In [128]:
returns = df.pct_change()
returns = returns.dropna()

In [129]:
#filter master dataframes based on date ranges
returns_all['short'] = returns.loc[short_start:short_end]
returns_benchmarks['short'] = returns.loc[short_start:short_end]
df_all['short'] = df.loc[short_start:short_end]
df_benchmarks['short'] = df.loc[short_start:short_end]

returns_all['med'] = returns.loc[med_start:med_end]
returns_benchmarks['med'] = returns.loc[med_start:med_end]
df_all['med'] = df.loc[med_start:med_end]
df_benchmarks['med'] = df.loc[med_start:med_end]

returns_all['long'] = returns.loc[long_start:long_end]
returns_benchmarks['long'] = returns.loc[long_start:long_end]
df_all['long'] = df.loc[long_start:long_end]
df_benchmarks['long'] = df.loc[long_start:long_end]

#drop coins that aren't benchmarks
for c in drop_list:
    for t in tfs:
        returns_benchmarks[t] = returns_benchmarks[t].drop([c] , axis=1)
        df_benchmarks[t] = df_benchmarks[t].drop([c] , axis=1)

In [130]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.128770,-0.081646
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.026888,0.018834
2019-02-27,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.005211,-0.005471
2019-02-28,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,-0.011987,-0.002390
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,0.005055,0.001741


In [131]:
#drop sectors that don't have data back to 1/1/2018
for i in long_drop:
    df_benchmarks['long'] = df_benchmarks['long'].drop([i] , axis=1)
    returns_benchmarks['long'] = returns_benchmarks['long'].drop([i] , axis=1)

In [132]:
#calculate sortino to btc
for t in tfs:
    sortino_df[t] = pd.DataFrame()
    sortino[t] = pd.DataFrame(index = ['sortino'])
    for col in returns_benchmarks[t]:
        sortino_df[t][col] = returns_benchmarks[t][col] - returns_benchmarks[t]['BTC']
        avg_ret = np.mean(sortino_df[t][col])
        a = [r ** 2 for r in sortino_df[t][col] if r < 0]
        a_sum = np.sum(a)
        b = len(sortino_df[t][col])
        down_risk = np.sqrt(a_sum/b)
        if down_risk > 0.000001:
            sortino[t][col] = (avg_ret / down_risk)*math.sqrt(365)
        else:
            sortino[t][col] = 0

In [133]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.128770,-0.081646
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.026888,0.018834
2019-02-27,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.005211,-0.005471
2019-02-28,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,-0.011987,-0.002390
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,0.005055,0.001741


In [134]:
data_coins_week = data_coins.loc[short_start:short_end]
#data_coins_week = data_coins_week[:]
net_week_return = data_coins_week.pct_change(len(data_coins_week)-1)
ann_week_factor = 365.000/(len(data_coins_week))
data_coins_week_returns = pd.DataFrame()
data_coins_week_returns = data_coins_week.pct_change()
data_coins_week_returns = data_coins_week_returns.dropna()
total_week_return = net_week_return.tail(1)*100
data_coins_week = data_coins_week.append(total_week_return)
data_coins_week.index = data_coins_week.index[:-1].append(pd.Index(['return']))

In [135]:
data_coins_week

,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
2019-02-01 00:00:00,106.940900,0.310080,0.081510,31.582380,113.855540,0.025150,2.320780,0.038370,63.784020
2019-02-02 00:00:00,107.600930,0.308830,0.081560,32.921310,115.791230,0.026350,2.375950,0.038890,64.382450
2019-02-03 00:00:00,109.470430,0.305300,0.081340,34.056510,119.396750,0.025570,2.391120,0.038550,64.364290
2019-02-04 00:00:00,106.533190,0.299450,0.079410,33.163810,117.557760,0.025170,2.358450,0.037850,63.267440
2019-02-05 00:00:00,106.857480,0.297330,0.078050,33.872030,117.918550,0.026700,2.385360,0.037690,62.616330
2019-02-06 00:00:00,106.494340,0.293810,0.075640,33.681330,115.625790,0.025780,2.344850,0.037340,60.059600
2019-02-07 00:00:00,104.071050,0.286960,0.073410,32.492230,113.693800,0.025270,2.330850,0.036370,62.074720
2019-02-08 00:00:00,106.282050,0.304710,0.077920,34.602420,117.799570,0.027300,2.454950,0.038490,65.547730
2019-02-09 00:00:00,118.499660,0.309310,0.080520,42.681750,128.417160,0.026500,2.698510,0.040350,66.661070
2019-02-10 00:00:00,119.028000,0.311240,0.080520,44.650460,127.399090,0.026190,2.794000,0.041060,65.832840


In [136]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.011593,-0.007133
2019-02-02,0.010041,0.011111,0.008623,0.010051,0.011235,0.004722,0.028551,0.030775,0.006172,0.006958
2019-02-03,0.015933,0.004272,0.006784,0.014922,0.014239,0.014904,0.012689,0.005754,0.017374,0.009043
2019-02-04,-0.020945,-0.015801,-0.003981,-0.020153,-0.018528,-0.023694,-0.009049,-0.013480,-0.026831,-0.013916
2019-02-05,-0.003668,0.000808,-0.013358,-0.003647,-0.004557,-0.007102,0.018634,0.026514,0.003044,-0.003536
2019-02-06,0.001495,0.017993,-0.017855,0.002038,0.002691,-0.002001,0.016667,0.009467,-0.003398,0.004093
2019-02-07,-0.017216,-0.016160,-0.016814,-0.017138,-0.015711,-0.025014,0.001640,-0.006391,-0.022755,-0.015612
2019-02-08,-0.005118,0.004567,0.007774,-0.004129,-0.005254,0.001828,-0.008564,-0.007530,0.021245,-0.003968
2019-02-09,0.099001,0.089244,0.074505,0.097697,0.093065,0.110015,0.069554,0.065363,0.114955,0.079441
2019-02-10,-0.001943,0.010255,0.007582,-0.000879,-0.002645,0.005028,0.015300,0.007550,0.004459,0.000439


In [137]:
print ('short start: ', returns_benchmarks['short'].index[0])
print ('medium start: ', returns_benchmarks['med'].index[0])
print ('long start: ', returns_benchmarks['long'].index[0])
print ('end: ', returns_benchmarks['short'].index[-2])
print ('end: ', returns_benchmarks['med'].index[-2])
print ('end: ', returns_benchmarks['long'].index[-2])

short start:  2019-02-01 00:00:00
medium start:  2019-01-01 00:00:00
long start:  2018-11-09 00:00:00
end:  2019-02-28 00:00:00
end:  2019-02-28 00:00:00
end:  2019-02-28 00:00:00


In [138]:
returns_benchmarks['short'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.011593,-0.007133
2019-02-02,0.010041,0.011111,0.008623,0.010051,0.011235,0.004722,0.028551,0.030775,0.006172,0.006958
2019-02-03,0.015933,0.004272,0.006784,0.014922,0.014239,0.014904,0.012689,0.005754,0.017374,0.009043
2019-02-04,-0.020945,-0.015801,-0.003981,-0.020153,-0.018528,-0.023694,-0.009049,-0.013480,-0.026831,-0.013916
2019-02-05,-0.003668,0.000808,-0.013358,-0.003647,-0.004557,-0.007102,0.018634,0.026514,0.003044,-0.003536


In [139]:
print (data_coins_week.head(n=1))
data_coins_week.tail(n=1)

                          ETH      XRP      XLM       LTC        BCH      TRX  \
2019-02-01 00:00:00  106.9409  0.31008  0.08151  31.58238  113.85554  0.02515   

                         EOS      ADA       BSV  
2019-02-01 00:00:00  2.32078  0.03837  63.78402  


,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,27.113995,1.119066,3.729604,44.293749,14.095072,-5.964215,51.014745,11.780036,4.309449


In [140]:
annualize = math.sqrt(365)
for t in tfs:
    results[t] = returns_benchmarks[t].describe()
    vol[t] = pd.DataFrame()
    vol[t] = results[t].loc['std']*annualize*100
    vol[t].name = 'annualized volatility'
    results[t] = results[t].append(df_benchmarks[t].iloc[0])
    results[t].index = results[t].index[:-1].append(pd.Index(['open price']))
    results[t] = results[t].append(df_benchmarks[t][-1:])
    results[t].index = results[t].index[:-1].append(pd.Index(['last price']))
    results[t] = results[t].append(vol[t])

In [141]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.128770,-0.081646
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.026888,0.018834
2019-02-27,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.005211,-0.005471
2019-02-28,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,-0.011987,-0.002390
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,0.005055,0.001741


In [142]:
for t in tfs:
    average_return[t] = pd.DataFrame()
    average_return[t] = returns_benchmarks[t]
    average_return[t].loc['mean'] = returns_benchmarks[t].mean(axis=0)
    returns_benchmarks[t] = returns_benchmarks[t][:-1]

In [143]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-02-01 00:00:00,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.011593,-0.007133
2019-02-02 00:00:00,0.010041,0.011111,0.008623,0.010051,0.011235,0.004722,0.028551,0.030775,0.006172,0.006958
2019-02-03 00:00:00,0.015933,0.004272,0.006784,0.014922,0.014239,0.014904,0.012689,0.005754,0.017374,0.009043
2019-02-04 00:00:00,-0.020945,-0.015801,-0.003981,-0.020153,-0.018528,-0.023694,-0.009049,-0.013480,-0.026831,-0.013916
2019-02-05 00:00:00,-0.003668,0.000808,-0.013358,-0.003647,-0.004557,-0.007102,0.018634,0.026514,0.003044,-0.003536
2019-02-06 00:00:00,0.001495,0.017993,-0.017855,0.002038,0.002691,-0.002001,0.016667,0.009467,-0.003398,0.004093
2019-02-07 00:00:00,-0.017216,-0.016160,-0.016814,-0.017138,-0.015711,-0.025014,0.001640,-0.006391,-0.022755,-0.015612
2019-02-08 00:00:00,-0.005118,0.004567,0.007774,-0.004129,-0.005254,0.001828,-0.008564,-0.007530,0.021245,-0.003968
2019-02-09 00:00:00,0.099001,0.089244,0.074505,0.097697,0.093065,0.110015,0.069554,0.065363,0.114955,0.079441
2019-02-10 00:00:00,-0.001943,0.010255,0.007582,-0.000879,-0.002645,0.005028,0.015300,0.007550,0.004459,0.000439


In [144]:
for t in tfs:
    net_return[t] = df_benchmarks[t].pct_change(len(df_benchmarks[t])-1)
    ann_factor[t] = 365.000/(len(df_benchmarks[t]))
    total_return[t] = net_return[t].tail(1)*100
    total_return_excel[t] = net_return[t].tail(1)
    percent_sd[t] = results[t].iloc[2,:]*100
    results[t] = results[t].append(percent_sd[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['daily volatility']))
    #print total_return_excel[t]
    results[t] = results[t].append(total_return_excel[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['return']))
    #print results[t]
    #print average_return[t]/(percent_sd[t]*0.01).transpose()
    Sharpe_values = (average_return[t].loc['mean']/(percent_sd[t]*0.01))*annualize
    Sharpe_values = Sharpe_values.rename('sharpe_values')
    results[t] = results[t].append(Sharpe_values)
    results[t].index = results[t].index[:-1].append(pd.Index(['Sharpe']))
    results[t] = results[t].append(sortino[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['Sortino']))
#print (results[t])

In [145]:
returns_benchmarks['med'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-01-01 00:00:00,-0.046381,-0.050800,-0.056148,-0.046939,-0.042739,-0.057864,-0.040122,-0.046808,-0.038211,-0.031067
2019-01-02 00:00:00,0.046762,0.043521,0.042677,0.046447,0.043137,0.051576,0.036109,0.039667,0.058320,0.027360
2019-01-03 00:00:00,0.033371,0.035948,0.027412,0.033396,0.022256,0.053596,0.031989,0.027348,0.083028,0.021259
2019-01-04 00:00:00,-0.041354,-0.043182,-0.034871,-0.041317,-0.036859,-0.054475,-0.028133,-0.029247,-0.027875,-0.025187
2019-01-05 00:00:00,0.018170,0.025959,0.024532,0.018870,0.013260,0.025122,0.048155,0.050412,0.040172,0.005201


In [146]:
for t in tfs:
    summary[t] = results[t].iloc[8:]
    corr[t] = returns_benchmarks[t].corr(method='spearman')
    betas[t] = returns_benchmarks[t].cov()
    beta_results[t] = pd.DataFrame()
    beta_results[t] = betas[t].iloc[0,:] / betas[t].iloc[3,0]
    corr[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_corr'+t+'.csv')
    summary[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_results'+t+'.csv')
beta_all = pd.DataFrame()
beta_all['MTD'] = beta_results['short']
beta_all['QTD'] = beta_results['med']
beta_all['YTD'] = beta_results['long']

In [147]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
2019-02-25 00:00:00,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.128770,-0.081646
2019-02-26 00:00:00,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.026888,0.018834
2019-02-27 00:00:00,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.005211,-0.005471
2019-02-28 00:00:00,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,-0.011987,-0.002390
2019-03-01 00:00:00,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,0.005055,0.001741


In [148]:
summary['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
open price,374.603942,313.478539,49.263316,156.644587,20.150812,13.096591,14.062167,43.673755,106.940900,3460.550000
last price,424.914158,381.398019,57.301770,178.648490,22.516762,15.764553,17.515368,50.875261,135.936850,3854.960000
annualized volatility,67.025544,62.029031,56.191178,66.002242,60.891152,82.038591,52.589428,52.100334,83.226787,52.580818
daily volatility,3.508277,3.246748,2.941181,3.454715,3.187189,4.294096,2.752656,2.727056,4.356289,2.752206
return,0.134302,0.216664,0.163173,0.140470,0.117412,0.203714,0.245567,0.164893,0.271140,0.113973
Sharpe,2.476499,3.594846,2.830471,2.564431,2.440604,2.849430,4.730093,2.936254,3.991633,2.755566
Sortino,1.874435,4.685765,0.843742,2.273459,0.515708,4.270734,5.519302,0.382424,9.322611,0.000000


In [149]:
summary['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
open price,420.511599,337.862280,54.644247,175.290083,22.237102,15.347474,13.168780,39.924454,134.063560,3746.710000
last price,424.914158,381.398019,57.301770,178.648490,22.516762,15.764553,17.515368,50.875261,135.936850,3854.960000
annualized volatility,65.777609,72.747511,67.634188,65.839665,59.762934,83.248551,71.696154,74.091912,83.157289,51.351689
daily volatility,3.442957,3.807779,3.540135,3.446206,3.128135,4.357428,3.752748,3.878148,4.352651,2.687870
return,0.010470,0.128856,0.048633,0.019159,0.012576,0.027176,0.330068,0.274288,0.013973,0.028892
Sharpe,-0.021956,0.949376,0.245105,0.053790,-0.026823,0.171712,2.466828,1.990653,0.225191,0.216776
Sortino,-0.959734,2.597007,0.247774,-0.568930,-1.498274,0.120103,7.039562,5.517780,0.237733,0.000000


In [150]:
summary['long']

,ten,twenty,forty,total,ERC20,ETH,BTC
open price,740.953740,637.900186,110.287135,311.303644,73.888661,216.812750,6522.270000
last price,424.914158,381.398019,57.301770,178.648490,50.875261,135.936850,3854.960000
annualized volatility,85.369336,96.667502,99.162240,86.135318,99.542633,98.773786,75.561797
daily volatility,4.468435,5.059808,5.190389,4.508529,5.210300,5.170056,3.955085
return,-0.426531,-0.402104,-0.480431,-0.426128,-0.311461,-0.373022,-0.408954
Sharpe,-1.690428,-1.243088,-1.644315,-1.664784,-0.719295,-1.048266,-1.883880
Sortino,-0.155548,0.941684,-0.775243,-0.080564,2.600611,1.172050,0.000000


In [151]:
corr['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
ten,1.000000,0.863711,0.843459,0.995621,0.991242,0.959496,0.765736,0.795840,0.894910,0.974275
twenty,0.863711,1.000000,0.710454,0.873016,0.859880,0.902573,0.876300,0.872469,0.779420,0.854406
forty,0.843459,0.710454,1.000000,0.851122,0.829228,0.879584,0.638752,0.653530,0.865353,0.830323
total,0.995621,0.873016,0.851122,1.000000,0.987958,0.966612,0.794745,0.820471,0.901478,0.973180
currency,0.991242,0.859880,0.829228,0.987958,1.000000,0.940339,0.777230,0.808976,0.874111,0.980296
platform,0.959496,0.902573,0.879584,0.966612,0.940339,1.000000,0.766284,0.783799,0.922277,0.936508
application,0.765736,0.876300,0.638752,0.794745,0.777230,0.766284,1.000000,0.986864,0.665025,0.789272
ERC20,0.795840,0.872469,0.653530,0.820471,0.808976,0.783799,0.986864,1.000000,0.680898,0.805145
ETH,0.894910,0.779420,0.865353,0.901478,0.874111,0.922277,0.665025,0.680898,1.000000,0.884510
BTC,0.974275,0.854406,0.830323,0.973180,0.980296,0.936508,0.789272,0.805145,0.884510,1.000000


In [152]:
corr['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,ETH,BTC
ten,1.000000,0.900468,0.893863,0.997779,0.994565,0.978317,0.817300,0.814670,0.825716,0.960666
twenty,0.900468,1.000000,0.830918,0.915371,0.895675,0.929632,0.932963,0.934775,0.748042,0.889538
forty,0.893863,0.830918,1.000000,0.902338,0.889714,0.914027,0.792577,0.775862,0.786207,0.859848
total,0.997779,0.915371,0.902338,1.000000,0.992051,0.982408,0.841379,0.838282,0.828755,0.962653
currency,0.994565,0.895675,0.889714,0.992051,1.000000,0.965459,0.817241,0.814027,0.808825,0.958796
platform,0.978317,0.929632,0.914027,0.982408,0.965459,1.000000,0.832437,0.826417,0.837580,0.947984
application,0.817300,0.932963,0.792577,0.841379,0.817241,0.832437,1.000000,0.987493,0.691584,0.813326
ERC20,0.814670,0.934775,0.775862,0.838282,0.814027,0.826417,0.987493,1.000000,0.682992,0.805669
ETH,0.825716,0.748042,0.786207,0.828755,0.808825,0.837580,0.691584,0.682992,1.000000,0.813326
BTC,0.960666,0.889538,0.859848,0.962653,0.958796,0.947984,0.813326,0.805669,0.813326,1.000000


In [153]:
corr['long']

,ten,twenty,forty,total,ERC20,ETH,BTC
ten,1.000000,0.931706,0.941427,0.998838,0.879301,0.849720,0.974434
twenty,0.931706,1.000000,0.918577,0.942213,0.952566,0.833635,0.920149
forty,0.941427,0.918577,1.000000,0.948372,0.884811,0.849241,0.924881
total,0.998838,0.942213,0.948372,1.000000,0.892490,0.853624,0.975057
ERC20,0.879301,0.952566,0.884811,0.892490,1.000000,0.771321,0.876815
ETH,0.849720,0.833635,0.849241,0.853624,0.771321,1.000000,0.804960
BTC,0.974434,0.920149,0.924881,0.975057,0.876815,0.804960,1.000000


In [93]:
beta_all

,MTD,QTD,YTD
ten,1.015897,0.999704,0.991443
twenty,0.870463,1.021342,1.078920
forty,0.791862,0.945079,1.100599
total,1.000000,1.000000,1.000000
currency,0.920786,0.902802,NaN
platform,1.225779,1.246156,NaN
application,0.683329,0.904314,NaN
ERC20,0.676321,0.913469,1.052608
BTC,0.217382,0.292723,0.597361
ETH,1.149069,1.118268,1.012332


In [94]:
coins_high_low = data_coins_week_returns.describe()
coins_high_low

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,0.004004,0.009526,0.000755,0.001825,0.014759,0.005599,-0.001540,0.016133,0.004368,0.002307
std,0.020192,0.043653,0.027404,0.032937,0.059017,0.042694,0.036700,0.052795,0.028227,0.041006
min,-0.045528,-0.128770,-0.056299,-0.066581,-0.089641,-0.127891,-0.104960,-0.120839,-0.054752,-0.089860
25%,-0.006086,-0.007437,-0.015914,-0.018777,-0.011366,-0.010999,-0.020927,-0.012953,-0.010544,-0.016156
50%,0.000901,0.005614,-0.003346,-0.000065,0.018738,-0.000734,-0.008355,0.014700,0.000457,-0.005020
75%,0.018194,0.022522,0.010873,0.009898,0.035546,0.022805,0.017152,0.029484,0.021341,0.020676
max,0.045063,0.114955,0.076641,0.087388,0.233490,0.126676,0.080332,0.179601,0.058290,0.102308


In [95]:
returns_benchmarks['short'].describe()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,0.004548,0.006109,0.004357,0.004637,0.004072,0.006404,0.006815,0.004191,0.003926,0.009102
std,0.035083,0.032467,0.029412,0.034547,0.031872,0.042941,0.027527,0.027271,0.019970,0.043563
min,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.045528,-0.128770
25%,-0.011183,-0.009923,-0.010825,-0.011053,-0.008193,-0.016901,-0.008685,-0.009017,-0.004727,-0.007437
50%,0.001248,0.004419,0.007192,0.002103,0.001441,0.002737,0.014040,0.005451,0.000901,0.005614
75%,0.016787,0.022655,0.019059,0.017700,0.014487,0.025802,0.018508,0.021403,0.016734,0.021368
max,0.099001,0.089244,0.074505,0.097697,0.093065,0.110015,0.069554,0.065363,0.045063,0.114955


In [96]:
data_coins_week.tail(n=1)

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,11.229919,27.113995,1.119066,3.729604,44.293749,14.095072,-5.964215,51.014745,11.780036,4.309449
